In [1]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#load corr matrix from file
import pickle
import numpy as np
all_corr = pickle.load(open('/content/drive/My Drive/correlation_matrix/correlations_filecc200.pkl', 'rb'))

In [3]:
#divide whole collection to normal and autistic
normals = {}
autistic = {}
for item in all_corr:
  if(all_corr[item][1] == 0):
    normals[item] = all_corr[item]
  else:
    autistic[item] = all_corr[item]

In [4]:
#sum over nomal
sum_n = np.zeros(19900)
for item_n in normals:
  sum_n = sum_n + normals[item_n][0]

#sum over autistic
sum_a = np.zeros(19900)
for item_a in autistic:
  sum_a = sum_a + autistic[item_a][0]

print(np.shape(sum_n))

(19900,)


In [5]:
#make diff matrix
diff = sum_n - sum_a
print(np.shape(diff))
print(sum_n)

(19900,)
[-68.60419034 -41.05806121 103.05643242 ...  13.22617837  12.7733547
 -25.46759707]


In [6]:
#add index to diff matrix
num_diff = []
for counter in range(len(diff)):
  num_diff = num_diff + [[counter ,diff[counter]]]

print(num_diff)

[[0, -13.710015531728473], [1, -10.724569746829943], [2, 22.194136879049623], [3, -1.5853319398899295], [4, -15.223664115835334], [5, -15.309622113155385], [6, -12.119616692594626], [7, 20.593747022031053], [8, 34.164394499453124], [9, -10.38536171902469], [10, -14.305287390370637], [11, -6.920251917217506], [12, 5.739596039094543], [13, 21.73198572562496], [14, 7.670960465076388], [15, -7.6931534157166865], [16, -0.33677882946337867], [17, -25.06185000723645], [18, 13.068254222459347], [19, -22.97317588028914], [20, 11.774478772275941], [21, 17.804365762603794], [22, -12.054835258989055], [23, -8.742243423243103], [24, 14.746373299499162], [25, -20.84193477152794], [26, 10.722037424055102], [27, -13.577574099899842], [28, -6.437151960201311], [29, -26.534069707980912], [30, -13.765844187820896], [31, 4.031538136050372], [32, -15.544749122093215], [33, -1.314179332556762], [34, 21.766951030264394], [35, -9.099965859714047], [36, -16.001400531285448], [37, -17.220208844594858], [38, -2.

In [7]:
#sort diff matrix
arr = np.array(num_diff)
sorted_diff = arr[arr[:,1].argsort()]

In [9]:
#make a mask from first and last 1000 elements
mask1 = sorted_diff[0:1000,0]
mask2 = sorted_diff[-1000:,0]
mask = np.concatenate((mask1, mask2))
mask = mask.astype('int32')
print(mask)

[ 4029 19820  7011 ... 14986  5835   155]


In [10]:
# apply mask to data
masked_diff = np.array(num_diff)[mask.astype(int)]
print(masked_diff)

[[ 4029.           -48.43271335]
 [19820.           -46.65522697]
 [ 7011.           -46.06028444]
 ...
 [14986.            65.99371075]
 [ 5835.            67.8990019 ]
 [  155.            68.95163944]]


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

X = []
for ff in all_corr:
  tempx = all_corr[ff][0]
  masked_tempx = np.array(tempx)[mask.astype(int)]
  X.append(list(masked_tempx))
X = pd.DataFrame(X)
sc = StandardScaler()
X = sc.fit_transform(X)
Y = []
for jj in all_corr:
  Y.append(all_corr[jj][1])
Y = np.asarray(Y)

print(np.shape(X))
print(np.shape(Y))

(884, 2000)
(884,)


In [12]:
import keras
classifier = Sequential()
classifier.add(Dense(100, activation='relu', kernel_initializer='random_normal', input_dim=2000))
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(2, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
loss_fn = tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')

classifier.compile(optimizer ='adam',loss=loss_fn, metrics =['accuracy'])

accuracy = []
F1 = []
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10)

In [13]:
classifier.fit(x_train,y_train, batch_size = 40, epochs = 100)
y_pred=classifier.predict(x_test)

y_pred = np.where(y_pred > 0.5, 1, 0)

cm = confusion_matrix(y_test, y_pred)
accuracy.append((cm[0][0] + cm[1][1])/sum(sum(cm)))
precision = cm[0][0]/(cm[0][0] + cm[1][1])
recall =  cm[0][0]/(cm[0][0] + cm[1][0])
F1.append(2 * precision * recall / (precision + recall))

print('accuracy =', np.mean(accuracy))
print('F1 =', np.mean(F1))

Epoch 1/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2497 - accuracy: 0.5434
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2460 - accuracy: 0.5434
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2305 - accuracy: 0.5434
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2121 - accuracy: 0.5434
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.1925 - accuracy: 0.6113
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1683 - accuracy: 0.8667
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1425 - accuracy: 0.8943
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1255 - accuracy: 0.9497
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1150 - accuracy: 0.9698
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.1089 - accuracy: 0.9748
Epoch 11/

In [14]:
y_pred = np.where(y_pred > 0.5, 1, 0)

cm = confusion_matrix(y_test, y_pred)
accuracy.append((cm[0][0] + cm[1][1])/sum(sum(cm)))
precision = cm[0][0]/(cm[0][0] + cm[1][1])
recall =  cm[0][0]/(cm[0][0] + cm[1][0])
F1.append(2 * precision * recall / (precision + recall))

print('accuracy =', np.mean(accuracy))
print('F1 =', np.mean(F1))


accuracy = 0.5393258426966292
F1 = 0.5161290322580646


In [15]:
print(cm)

[[24 20]
 [21 24]]
